# 0. 전처리
---

- 구글 드라이브를 마운트하는 코드이다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



> **!!!필수 데이터셋 경로를 지정해주는 코드이다.**  (개인의 환경에 따라 수정 필요)



In [ ]:
dataset_dir = "./DDC_dataset"

## 0.1. 최초 세팅

- 데이터셋 파일의 압축을 푸는 코드이다.
  - 파일은 구글 드라이브의 Root에 위치해야 하며, 이름은 "2024 데이터 크리에이터 캠프 대학부 데이터셋.zip"이어야 함.

In [ ]:
!unzip -qq "./drive/MyDrive/2024 데이터 크리에이터 캠프 대학부 데이터셋.zip" -d "/drive/MyDrive/DCC_dataset/"

In [ ]:
import zipfile
import os

# 압축 파일 경로 설정
zip_file_path = '/content/drive/MyDrive/2024 데이터 크리에이터 캠프 대학부 데이터셋.zip'

# 압축을 풀 폴더 경로
extract_folder = '/content/drive/MyDrive/DCC_dataset'

# 압축 풀기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

- 대용량의 데이터셋 압축 파일을 저장하고, 해제하는데 어려움이 있었음
- 이에 이름의 형식은 같은 더미데이터를 사용하기로 결정함
- 아래는 더미데이터를 만들어내는 코드

In [ ]:
# 더미 데이터 생성 코드
import random
import os


# 설정: 생성할 파일의 개수와 저장할 경로
num_files = 50
output_dir = "./dummy_files/"

# 파일을 저장할 디렉터리 생성 (존재하지 않으면 생성)
os.makedirs(output_dir, exist_ok=True)

# 스타일 및 성별 리스트
styles = ["feminine", "classic", "minimal", "lounge", "popart", "normcore", "ivy", "mods", "hippie", "bold"]
genders = ["W", "M"]  # W: 여성, M: 남성
fabrics = ["W", "T"] # W: 울, T: 테트론, 폴리에스테르
# 더미 파일 생성
for _ in range(num_files):
    # 원단, 이미지ID, 시대, 스타일, 성별 임의로 선택
    fabric = random.choice(fabrics) # 원단 중 임의 선택
    image_id = random.randint(10000, 99999)  # 이미지ID는 10000~99999 사이의 숫자
    era = random.randint(19, 60)  # 시대는 19~60 사이의 숫자
    style = random.choice(styles)  # 스타일 임의 선택
    gender = random.choice(genders)  # 성별 임의 선택


    # 파일명 생성
    file_name = f"{fabric}_{image_id}_{era}_{style}_{gender}.jpg"

    # 더미 파일 생성
    with open(os.path.join(output_dir, file_name), 'w') as f:
        f.write("테스트용 더미 데이터 파일입니다.")

# 결과 출력
print(f"{num_files}개의 더미 파일이 '{output_dir}' 디렉터리에 생성되었습니다.")

50개의 더미 파일이 './dummy_files/' 디렉터리에 생성되었습니다.


# Mission 1. 패션 스타일 이미지 분류
---

## 1-1. 파일명 기반 성별&스타일 시각화
> 주어진 이미지 데이터의 파일명에 기반하여 성별&스타일 통계치를 표 형식으로 기입하라.

### 1-1.1. Training 이미지 파일명 기반 시각화

- 정규 표현식을 사용하여 Training Image 파일명에서 성별과 스타일을 추출하여 데이터 프레임에 저장

In [ ]:
import os
import re
import pandas as pd
from collections import Counter

# 파일 불러오기
training_image_files = os.listdir(f'{dataset_dir}/training_image/')

# 정규표현식을 사용하여 파일명에서 성별과 스타일을 추출
pattern = re.compile(r'(W|T)_(\d+)_(\d+)_(\w+)_(M|W)\.jpg')

# 성별과 스타일 카운트
counts = Counter()

for file_name in training_image_files:
    match = pattern.match(file_name)
    if match:
        style = match.group(4)  # 스타일
        gender = '여성' if match.group(5) == 'W' else '남성'  # 성별
        counts[(gender, style)] += 1

# 데이터 프레임 생성
df = pd.DataFrame(counts.items(), columns=['성별 & 스타일', '이미지 수'])
df[['성별', '스타일']] = pd.DataFrame(df['성별 & 스타일'].tolist(), index=df.index)
df.drop(columns=['성별 & 스타일'], inplace=True)
df.sort_values(by='성별', inplace=True)

print(df)

    이미지 수  성별             스타일
0     269  남성            mods
19    268  남성            bold
29    298  남성  sportivecasual
12    364  남성        normcore
11    260  남성          hippie
15    237  남성             ivy
3     274  남성          hiphop
2     278  남성     metrosexual
6     157  여성  sportivecasual
28     77  여성      genderless
27     48  여성          hiphop
26     37  여성           disco
25     41  여성          popart
24     37  여성           space
23     78  여성        oriental
22     91  여성          kitsch
21     91  여성          hippie
20     45  여성          lounge
1      55  여성        lingerie
18    120  여성       powersuit
17     67  여성      athleisure
16     67  여성        cityglam
14    153  여성        normcore
13     64  여성         ecology
4     154  여성        feminine
10     95  여성   bodyconscious
9      33  여성        military
8      77  여성         classic
7      65  여성            punk
5     139  여성         minimal
30     31  여성          grunge


- 저장된 데이터 프레임을 시각화

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# 성별에 따라 색상을 지정하는 함수
def get_fill_colors(genders):
    fill_colors = []
    for gender in genders:
        if gender == '남성':
            fill_colors.append('rgb(203, 230, 211)')
        elif gender == '여성':
            fill_colors.append('rgb(225, 233, 246)')
        else:
            fill_colors.append('white')
    return fill_colors

# Plotly 테이블 생성
fig = go.Figure(data=[go.Table(
    columnwidth=[80, 100, 80],  # 각 열의 너비를 설정
    header=dict(values=['성별', '스타일', '이미지 수'],
                fill_color='rgb(33, 107, 171)',
                font_color='white',
                align='center',
                font_size=27,
                height=40),

    cells=dict(values=[df['성별'], df['스타일'], df['이미지 수']],
               fill_color=[get_fill_colors(df['성별']), 'white', 'white'], # 성별의 배경 색상을 다르게 지정, 스타일, 이미지 수의 배경은 그대로
               line_color='rgb(33, 107, 171)',
               font_color='rgb(33, 107, 171)',
               font_size=20,
               align='center',
               height=30))])
# 제목 추가
fig.update_layout(
    title={
        'text': "[Training] 성별과 스타일에 따른 이미지 수",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_color': 'rgb(33, 107, 171)',
        'font_size': 30
    }
)
# 테이블 출력
fig.show()


### 1-1.2 Validation 이미지 파일명 기반 시각화

- 정규 표현식을 사용하여 Validation Image 파일명에서 성별과 스타일을 추출하여 데이터 프레임에 저장

In [ ]:
import os
import re
import pandas as pd
from collections import Counter

# 파일 불러오기
training_image_files = os.listdir(f'{dataset_dir}/validation_image/')

# 정규표현식을 사용하여 파일명에서 성별과 스타일을 추출
pattern = re.compile(r'(W|T)_(\d+)_(\d+)_(\w+)_(M|W)\.jpg')

# 성별과 스타일 카운트
counts = Counter()

for file_name in training_image_files:
    match = pattern.match(file_name)
    if match:
        style = match.group(4)  # 스타일
        gender = '여성' if match.group(5) == 'W' else '남성'  # 성별
        counts[(gender, style)] += 1

# 데이터 프레임 생성
df = pd.DataFrame(counts.items(), columns=['성별 & 스타일', '이미지 수'])
df[['성별', '스타일']] = pd.DataFrame(df['성별 & 스타일'].tolist(), index=df.index)
df.drop(columns=['성별 & 스타일'], inplace=True)
df.sort_values(by='성별', inplace=True)

print(df)

- 저장된 데이터 프레임을 시각화

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# 성별에 따라 색상을 지정하는 함수
def get_fill_colors(genders):
    fill_colors = []
    for gender in genders:
        if gender == '남성':
            fill_colors.append('rgb(203, 230, 211)')
        elif gender == '여성':
            fill_colors.append('rgb(225, 233, 246)')
        else:
            fill_colors.append('white')
    return fill_colors

# Plotly 테이블 생성
fig = go.Figure(data=[go.Table(
    columnwidth=[80, 100, 80],  # 각 열의 너비를 설정
    header=dict(values=['성별', '스타일', '이미지 수'],
                fill_color='rgb(33, 107, 171)',
                font_color='white',
                align='center',
                font_size=27,
                height=40),

    cells=dict(values=[df['성별'], df['스타일'], df['이미지 수']],
               fill_color=[get_fill_colors(df['성별']), 'white', 'white'], # 성별의 배경 색상을 다르게 지정, 스타일, 이미지 수의 배경은 그대로
               line_color='rgb(33, 107, 171)',
               font_color='rgb(33, 107, 171)',
               font_size=20,
               align='center',
               height=30))])
# 제목 추가
fig.update_layout(
    title={
        'text': "[Validation] 성별과 스타일에 따른 이미지 수",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_color': 'rgb(33, 107, 171)',
        'font_size': 30
    }
)
# 테이블 출력
fig.show()


## 1-2. ResNet-18을 사용한 성별&스타일 클래스 분류
> ResNet-18을 활용하여 성별&스타일 단위로 클래스 분류를 수행하고 Validation 데이터에 대한 정확도(Top-1 Accuracy)를 제시하라.

In [ ]:
## 라이브러리 임포트
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from torch.backends import cudnn
cudnn.benchmark = True

import torchvision
from torchvision import models
from torchvision import transforms as T
from torchvision.datasets import ImageFolder

from torchsummary import summary
from tqdm import tqdm
from tqdm.notebook import tqdm
from datetime import datetime
import os

from warnings import filterwarnings
filterwarnings('ignore')

import pickle

In [ ]:
# 데이터를 원하는 스타일&성별로 class를 만들어 분류할 수 있는 dataset으로 만드는 과정
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

def extract_label(filename):
  parts = filename.split("_")
  style = parts[3]
  gender = parts[4].split(".")[0]
  return f"{style}_{gender}"


class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith(".jpg")]
        self.transform = transform

        self.labels = [extract_label(f) for f in self.image_files]
        unique_labels = sorted(list(set(self.labels)))

        # classes 속성 추가
        self.classes = unique_labels  # 고유한 레이블 목록
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
        self.indexed_labels = [self.label_to_idx[label] for label in self.labels]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        label = self.indexed_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms


# 데이터 전처리
transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.54984684, 0.52261154, 0.50516839], std=[0.26895054, 0.26690443, 0.27051458])
  ])

# 데이터셋 준비
train_image = '/content/drive/MyDrive/DCC_dataset/training_image'
test_image = '/content/drive/MyDrive/DCC_dataset/validation_image'

train_dataset = CustomImageDataset(train_image, transform=transform)
test_dataset = CustomImageDataset(test_image, transform=transform)

train_loader = DataLoader(dataset = train_dataset, batch_size = 30, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size= 30, shuffle=False)

print(f"Train set size:{len(train_dataset)}, Test set size: {len(test_dataset)}")
print(f"labels: {len(set(train_dataset.labels))}")
print(f"labels: {len(set(test_dataset.labels))}")

Train set size:4070, Test set size: 951
labels: 31
labels: 31


In [ ]:
def save_pickle(data, route):
    '''
    data를 받아서 pickle 형식으로 저장
    '''
    with open(route, 'wb') as file:
        pickle.dump(data, file)
    print(f"{route} 저장 완료")

def load_pickle(route):
    '''
    저장된 pickle 파일을 불러오기
    '''
    with open(route, 'rb') as file:
        return pickle.load(file)

In [ ]:
def save_scores(scores):
    '''
    epoch별 train accuracy와 validation accuracy를 pickle 파일로 저장
    '''
    with open('scores.pickle', 'wb') as file:
        pickle.dump(scores, file)

def load_scores(route):
    '''
    epoch별 train accuracy와 validation accuracy가 저장된 파일을 불러오기
    '''
    with open(route, 'rb') as file:
        return pickle.load(file)

In [ ]:
def save_model(model, optimizer, num_epochs, loss, accuracy_score):
    '''
    모델의 checkpoint를 '*.pt' 확장자 파일로 저장하는 함수
    '''
    # 모델을 저장할 경로를 지정하고 .pt 확장자를 사용
    model_save_path = 'Mission1.pt'

    # 모델의 상태 및 다른 중요한 정보를 저장
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': num_epochs,  # 현재 에폭
        'loss': loss.item()  # 현재 손실
    }

    torch.save(checkpoint, model_save_path)

def load_model(model, optimizer, model_path):
    '''
    저장된 모델 checkpoint를 불러오는 함수
    '''
    checkpoint = torch.load(model_path)

    # 모델의 상태 및 중요한 정보를 불러옴
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return model, optimizer, epoch, loss

# 저장 경로는 때에 이용 목적에 따라 바꾸도록 한다

In [ ]:
import torch
import torch.nn as nn  # nn 모듈을 임포트
# 본인의 코드에 사용해도 된다.
@torch.no_grad()
def validate_epoch(model: nn.Module, test_loader: torch.utils.data.dataloader, device: torch.device):
    ''' data_loader provides inputs and GTs.
        model receives input images from data_loader and produces logits.
        returns the accuracy of model in percent.
    '''
    model.eval()
    accuracies = []
    for images, labels in tqdm(test_loader, total=len(test_loader), mininterval=1, desc='measuring accuracy'):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        pred = torch.argmax(logits, dim=1)
        accuracies.append((pred == labels).float().mean())

    accuracy = torch.stack(accuracies).float().mean() * 100
    return accuracy.item()

In [ ]:
# 클래스별 채점 코드
@torch.no_grad()
def validate_epoch_by_class(model: nn.Module, test_loader: torch.utils.data.dataloader, device: torch.device):
    ''' data_loader provides inputs and GTs.
        model receives input images from data_loader and produces logits.
        returns the accuracy of model BY CLASSES in percent.
    '''
    # 초기 설정
    total_counts = {test_loader.dataset.classes[i] : 0 for i in range(len(test_loader.dataset.classes))}
    true_counts = {test_loader.dataset.classes[i] : 0 for i in range(len(test_loader.dataset.classes))}

    # 클래스별 채점 수행
    model.eval()
    for images, labels in tqdm(test_loader, total=len(test_loader), mininterval=1, desc='measuring accuracy'):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        pred = torch.argmax(logits, dim=1)

        results = pred == labels

        # tensor를 구성하는 각 원소의 라벨과 결과를 확인하여, 클래스별 정답 개수와 데이터 개수로 저장
        for i in range(len(results)):
            total_counts[test_loader.dataset.classes[labels[i].item()]] += 1
            true_counts[test_loader.dataset.classes[labels[i].item()]] += 1 if results[i].item() else 0

    # 데이터프레임으로 생성
    df = pd.merge(
        pd.DataFrame(total_counts, index=['total']).T.reset_index(),
        pd.DataFrame(true_counts, index=['true']).T.reset_index()
    )

    # accuracy 도출
    df['accuracy'] = df['true'] / df['total']

    return df.sort_values('accuracy', ascending=False).reset_index(drop=True)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/')

In [ ]:
import torchvision.transforms as T
import torch
from torch.utils.data import DataLoader

# 데이터 전처리
transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.54984684, 0.52261154, 0.50516839], std=[0.26895054, 0.26690443, 0.27051458])
  ])

# 데이터셋 준비
train_image = '/content/drive/MyDrive/DCC_dataset/training_image'
test_image = '/content/drive/MyDrive/DCC_dataset/validation_image'

train_dataset = CustomImageDataset(train_image, transform=transform)
test_dataset = CustomImageDataset(test_image, transform=transform)

train_loader = DataLoader(dataset = train_dataset, batch_size = 128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size= 128, shuffle=False)

In [ ]:
# ResNet-18 모델 로딩
# model = models.resnet18(pretrained=False)

# epoch 16부터 모델을 불러오기
model_path = '/content/drive/MyDrive/model_save/model_epoch_16.pth'
model = model.load_state_dict(torch.load(model_path))  # 가중치 불러오기

# 분류할 클래스 수에 맞게 모델의 출력 레이어 수정
num_classes = len(train_dataset.labels)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# CUDA 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

AttributeError: '_IncompatibleKeys' object has no attribute 'to'

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# 모델 로딩 및 출력 레이어 크기 맞추기
model = models.resnet18(pretrained=False)
num_classes = len(train_dataset.labels)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 가중치 로드
model_path = '/content/drive/MyDrive/model_save/model_epoch_41.pth'
model.load_state_dict(torch.load(model_path))

# CUDA 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)


cpu


In [ ]:
# 시드 설정
seed = 42
torch.manual_seed(seed)  # PyTorch 무작위 시드 설정
np.random.seed(seed)     # NumPy 무작위 시드 설정

# GPU 시드 설정
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

### 모델 학습

In [ ]:
# 손실 함수 정의
criterion = torch.nn.CrossEntropyLoss()

# 최적화 알고리즘 선택
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

# 스케줄러 정의
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

# 훈련 반복
max_val_accuracy = 0
num_epochs = 50
scores = {}
for epoch in range(30, num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / (i + 1)
    accuracy = 100.0 * correct / total
    print(f'Epoch {epoch+1} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

    # 매 epoch에서 validation accuracy 계산
    val_accuracy = validate_epoch(model, test_loader, device)

    # accuracy를 저장
    scores[epoch] = [accuracy, val_accuracy]
    if val_accuracy > max_val_accuracy:
        max_val_accuracy = val_accuracy
        save_model(model, optimizer, epoch, loss, val_accuracy)

    # 모델 저장 (매 epoch마다)
    model_save_path = f'/content/drive/MyDrive/model_save/'
    model_file_path = os.path.join(model_save_path, f'model_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), model_file_path)
    print(f'Model saved at {model_file_path}')

    if val_accuracy > max_val_accuracy:
        max_val_accuracy = val_accuracy
        save_model(model, optimizer, epoch, loss, val_accuracy)

    # 스케줄러 업데이트
    scheduler.step()

print('Training Finished')

now = datetime.now().strftime('%y%m%d%H%M%S')
save_route = f"{now}_scores.pickle"
save_pickle(scores, save_route)


Epoch 31/50:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 31 - Loss: 0.2547, Accuracy: 92.43%


measuring accuracy:   0%|          | 0/8 [00:00<?, ?it/s]

Model saved at /content/drive/MyDrive/model_save/model_epoch_31.pth


Epoch 32/50:   0%|          | 0/32 [00:00<?, ?it/s]

...

measuring accuracy:   0%|          | 0/8 [00:00<?, ?it/s]

Model saved at /content/drive/MyDrive/model_save/model_epoch_41.pth


Epoch 42/50:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 42 - Loss: 0.0059, Accuracy: 99.78%


measuring accuracy:   0%|          | 0/8 [00:00<?, ?it/s]

### Validation 정확도 확인

In [ ]:
# 모델과 옵티마이저를 초기화
model = models.resnet18(pretrained=False)
num_classes = len(train_dataset.labels)
model.fc = nn.Linear(model.fc.in_features, num_classes)
optimizer = torch.optim.Adam(model.parameters())

# 저장된 모델을 불러오기
model_path = '/content/drive/MyDrive/model_save/model_epoch_40.pth'
model.load_state_dict(torch.load(model_path))  # 가중치 불러오기

# CUDA 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

# 시드 설정
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# GPU 시드 설정
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# 평가 모드 전환
model.eval()

cpu


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
validate_epoch(model, test_loader, device)

measuring accuracy:   0%|          | 0/8 [00:00<?, ?it/s]

62.08984375



*   

*   

*   

*   항목 추가

*   항목 추가

*   항목 추가
*   항목 추가

*   항목 추가

*   항목 추가
*   항목 추가


*   항목 추가




*   항목 추가


*   항목 추가


*   항목 추가


*   항목 추가


*   항목 추가




